# A joint model CAR (model 1) 
## Two independent processes
 $$ [P , S ] = [P ] [ S ] $$

In [1]:
## Import code:
setwd('/apps/external_plugins/biospytial_rwrapper/biospytial.rwrapper/R/')

In [4]:
rm(list=ls())                                                                                     
source("init_data.R")                                                                             
# load the building function                                                                      
source("joint.binomial.bymCARModel1.R")                                                                 
n.sample = 10000                                                                                  
burnin=10000                                                                                      
postburnin = burnin +1000                                                                         
thin = 1                                                                                          
verbose = TRUE                           
results  <- joint.binomial.bymCARModel1(formula_S = formula_sample, 
                                        formula_P = formula_presence,
                                        n.sample=n.sample,
                                        burnin=burnin,
                                        postburnin=postburnin,
                                        thin=thin,
                                        verbose=TRUE) 


Setting up the model.
Performing burnin period for Sample Effort Model 
  |======================================================================| 100%
Summarising results.
Finished in  79.6 seconds.
Setting up the model.
Performing burnin period for Presence model 
  |======================================================================| 100%
Summarising results.
Finished in  78.7 seconds.
Sampling from the joint model 10000 post burnin and thinned (if requested) samples.
  |======================================================================| 100%
[1] "Compiling summary for S process"
[1] "Compiling summary for the P process"


In [5]:
post.sample_p = data.frame(results$S$samples$fitted)
post.presence_p = data.frame(results$P$samples$fitted)

In [6]:
pss  <- post.sample_p %>% mutate_all(function(p) rbernoulli(1,p))
pps <- post.presence_p %>% mutate_all(function(p) rbernoulli(1,p))                                        

In [7]:

sumpps <- colSums(pps)

In [8]:
sumpss <- colSums(pss)

In [9]:
prob_pps <- sumpps / length(pps)

In [10]:
prob_pss <- sumpss / length(pss)

In [11]:
probpres = prob_pps * prob_pss

In [12]:
## save to posterity
# Save output sample to display in python
## Save the complete chain trace for analysing in Python,
TDF = read.csv("/outputs/training_data_sample_puebla_p9_abies_pinophyta.csv")
## Order it according to the id of the cell
## This is important because the adjancy matrix rows need to be the same
TDF = TDF[order(TDF$cell_ids),]

In [13]:
chain = cbind(TDF$cell_ids,probpres,prob_pps,prob_pss)
file_ = '/outputs/presence_only_models/modelCAR_model1_fitted.csv'
write.csv(chain,file_)